In [2]:
from disambiguation.core import Database
from disambiguation import config
from cPickle import load
import pandas as pd
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
retriever = Database.FecRetriever('delaware_combined')
retriever.retrieve()

select NAME,CONTRIBUTOR_ZIP,ZIP_CODE,CONTRIBUTOR_STREET_1,CITY,STATE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,CMTE_ID,ENTITY_TP,id from delaware_combined;


In [5]:
list_records = retriever.getRecords()

In [20]:
sel = retriever
# idm = Database.IdentityManager('USA')
# idm.fetch_dict_id_2_identity()

for r in sel.list_of_records[:1000]:
    r_id = r.id
    print r_id, idm.get_identity(r_id)




5785973 None
2987302 None
2637468 None
12175659 DE-7
11691599 DE-7
8849930 DE-7
8063706 DE-7
8828562 DE-7
11469405 DE-7
13557301 DE-7
12332345 DE-7
12014624 DE-7
12856234 DE-7
10852794 DE-7
10761407 DE-7
13520434 DE-7
13760544 DE-7
14181669 DE-7
8813365 DE-7
14288552 DE-7
6001878 DE-6
2919196 DE-6
11935249 DE-9
3857530 DE-9
9655062 DE-8
4229788 DE-4
7247268 DE-4
6558915 DE-4
3190265 DE-4
7236142 DE-4
11669429 DE-4
10398054 DE-4
12943844 DE-5
14115202 DE-5
5840102 DE-10
11179887 DE-11
10370939 DE-12
8394761 DE-15
9785941 DE-16
13701739 DE-16
9537235 DE-14
11426356 DE-14
2807433 DE-18
6321671 DE-17
802664 DE-13
10221956 DE-17
14422055 DE-17
8635278 DE-19
8579487 DE-19
9345009 DE-19
8450605 DE-19
12938808 DE-19
13032235 DE-19
5749881 DE-20
5353779 DE-20
4927750 DE-20
10965751 DE-21
12420861 DE-22
1457050 DE-23
566003 DE-23
3546960 DE-23
10774279 DE-23
7178581 DE-26
3841473 DE-24
3037815 DE-24
6315551 DE-24
7258921 DE-24
6617717 DE-24
4508242 DE-24
5735271 DE-24
4833752 DE-24
7076613 DE-24

In [21]:
df = pd.DataFrame()
df.to_csv()

In [24]:
' '.join(list(set([None])))

TypeError: sequence item 0: expected string, NoneType found

In [26]:
a = set([None])
b = {x for x in a if x}
' '.join(list(b))

''